In [3]:
import sys
import time

# === ADIM 1: KURULUM ===
print("--- ADIM 1: Kütüphaneler kuruluyor... ---")
# Self-Querying için 'langchain-chroma' ve 'lark' ekliyoruz
!pip install -q -U google-generativeai langchain-google-genai langchain faiss-cpu langchain_community sentence-transformers langchain-chroma lark
print("✅ Kütüphaneler kuruldu.")

# === ADIM 2: API ANAHTARI VE IMPORTLAR ===
print("\n--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---")
import os
from google.colab import userdata
import google.generativeai as genai

# Gerekli LangChain importları
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
# YENİ: Vektör veritabanı olarak Chroma
from langchain_chroma import Chroma
# YENİ: SelfQueryRetriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

try:
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
    print("✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!")
except Exception as e:
    print(f"❌ HATA: API Anahtarı yüklenemedi. 'Secrets' (🔑) bölümünü kontrol edin. Hata: {e}")
    sys.exit("API Anahtarı hatası.")

# === ADIM 3: GÜNCEL VERİYİ ÇEKME ===
print("\n--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---")
!rm -rf akbank-genai-bootcamp-proje
!git clone https://github.com/mustafadevrim/akbank-genai-bootcamp-proje
print("✅ Güncel repo klonlandı.")

# === ADIM 4: KONTEKST ENJEKSİYONU PARÇALAMASI ===
print("\n--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---")
try:
    with open("akbank-genai-bootcamp-proje/tarifler.txt", "r", encoding="utf-8") as f:
        tum_tarifler_metni = f.read()

    tarif_listesi = tum_tarifler_metni.split("\n---\n")
    print(f"{len(tarif_listesi)} adet ana tarif bulundu.")

    documents = [] # Parçaları (chunk) tutacak son liste

    for tarif_metni in tarif_listesi:
        if not tarif_metni.strip(): continue
        parts = tarif_metni.split("\nMalzemeler:\n", 1)
        if len(parts) < 2: continue
        baslik_content = parts[0].strip() # "Başlık: Karnıyarık"
        parts2 = parts[1].split("\nYapılışı:\n", 1)
        if len(parts2) < 2: continue
        malzemeler_content = parts2[0].strip()
        yapilis_content = parts2[1].strip()

        # Self-Querying için metadata'yı DÜZGÜN tanımlıyoruz
        # 'source' alanı "Başlık: Karnıyarık" metnini içerecek
        doc_metadata = {"source": baslik_content}

        # Stratejimiz (3 parça oluşturma) aynı
        documents.append(Document(page_content=baslik_content, metadata=doc_metadata))
        documents.append(Document(page_content=f"{baslik_content}\nMalzemeler:\n{malzemeler_content}", metadata=doc_metadata))
        documents.append(Document(page_content=f"{baslik_content}\nYapılışı:\n{yapilis_content}", metadata=doc_metadata))

    print(f"✅ Veri başarıyla {len(documents)} adet KONTEKSTLİ parçaya (chunk) bölündü.")

except Exception as e:
    print(f"❌ HATA: Veri yükleme veya parçalama hatası: {e}")
    sys.exit("Veri yükleme hatası.")

# === ADIM 5: EMBEDDING VE VEKTÖR DB (Chroma) ===
print("\n--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB (Chroma) oluşturuluyor... ---")
print("Lütfen bekleyin, 'MiniLM' modeli (yaklaşık 450 MB) indirilecek...")
start_time = time.time()

# 'mpnet' (kafa karıştıran) yerine 'MiniLM' (hızlı ve yeterli) modelini kullanıyoruz
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)
print(f"✅ Embedding modeli ({model_name}) yüklendi.")

# FAISS yerine Chroma veritabanını kullanıyoruz
vector_store = Chroma.from_documents(documents, embeddings)
print("✅ Vektör veritabanı (Chroma) 'MiniLM' ve 'Konteks Enjeksiyonu' ile başarıyla oluşturuldu!")
end_time = time.time()
print(f"(Bu işlem {end_time - start_time:.2f} saniye sürdü)")

# === ADIM 6: SELF-QUERYING RETRIEVER KURULUMU ===
print("\n--- ADIM 6: Self-Querying Retriever kuruluyor... ---")

# LLM (Gemini) modelini tanımla
llm = ChatGoogleGenerativeAI(model="models/gemini-flash-latest", temperature=0)

# Retriever'a metadata'mızın ne anlama geldiğini öğretiyoruz
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="Tarifin başlığı, örneğin 'Başlık: Karnıyarık' veya 'Başlık: Menemen'",
        type="string",
    ),
]
document_content_description = "Türk mutfağı yemek tarifleri"

# Retriever'ı oluştur
# Bu, LLM'i (Gemini) ve Vektör DB'yi (Chroma) birbirine bağlar
try:
    retriever = SelfQueryRetriever.from_llm(
        llm,
        vector_store,
        document_content_description,
        metadata_field_info,
        verbose=True # ÖNEMLİ: Gemini'nin filtreyi nasıl oluşturduğunu görmek için
    )
    print("✅ Self-Querying Retriever başarıyla kuruldu.")
except Exception as e:
    print(f"❌ HATA: Self-Querying Retriever kurulamadı: {e}")
    sys.exit("Retriever hatası.")

# === ADIM 7: RAG PIPELINE VE NİHAİ TEST ===
print("\n--- ADIM 7: RAG Pipeline kuruluyor ve test ediliyor... ---")

rag_pipeline = RetrievalQA.from_chain_type(
    llm=llm, # Cevap üretecek model (Gemini)
    chain_type="stuff",
    retriever=retriever # Akıllı filtreleme yapan retriever
)
print("✅ RAG Pipeline (Self-Querying ile) başarıyla kuruldu.")

print("\n--- NİHAİ TEST (Self-Querying ile) BAŞLIYOR ---")
soru_final = "Karnıyarık nasıl yapılır? Malzemeleri nelerdir?"
print(f"Soru: {soru_final}")

try:
    cevap = rag_pipeline.invoke(soru_final)
    # Gemini'nin oluşturduğu filtreyi ve bulunan belgeleri göreceğiz (verbose=True sayesinde)
    print("\nCevap:")
    print(cevap['result'])
except Exception as e:
    print(f"❌ HATA: Model sorgulanamadı. Hata: {e}")

--- ADIM 1: Kütüphaneler kuruluyor... ---
✅ Kütüphaneler kuruldu.

--- ADIM 2: Kütüphaneler ve API Anahtarı yükleniyor... ---
✅ API Anahtarı ve Kütüphaneler Başarıyla Yüklendi!

--- ADIM 3: GitHub'dan güncel veri seti çekiliyor... ---
Cloning into 'akbank-genai-bootcamp-proje'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 47.52 KiB | 1.25 MiB/s, done.
Resolving deltas: 100% (7/7), done.
✅ Güncel repo klonlandı.

--- ADIM 4: Veri yükleniyor ve KONTEKST ENJEKSİYONU ile parçalanıyor... ---
10 adet ana tarif bulundu.
✅ Veri başarıyla 30 adet KONTEKSTLİ parçaya (chunk) bölündü.

--- ADIM 5: Embedding modeli yükleniyor ve Vektör DB (Chroma) oluşturuluyor... ---
Lütfen bekleyin, 'MiniLM' modeli (yaklaşık 450 MB) indirilecek...
✅ Embedding modeli (sentence-transformers/paraphrase-multilingual-MiniLM-L1